In [209]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools
import networkx as nx
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import math

load the data

In [149]:
homes_B = pd.read_pickle("data/B_homes.pickle")
homes_G = pd.read_pickle("data/G_homes.pickle")

names_edges = ["from_id", "to_id"]

edges_B = pd.read_table("Brightkite_edges.txt", names=names_edges).dropna()
edges_G = pd.read_table("Gowalla_edges.txt", names=names_edges).dropna()

countries = pd.read_pickle("data/countries.pkl")

homes_B

,longitude,latitude,country
,mean,mean,
user_id,,,
0,-105.009140,39.756108,US
1,-122.353819,37.581821,US
2,-105.041977,39.771737,US
3,-122.424280,37.765768,US
4,24.937299,60.173985,FI
...,...,...,...
58222,35.833333,33.833333,LB
58224,35.833333,33.833333,LB


add religions

remove bidirection

In [150]:
edges_B[["from_id", "to_id"]] = np.sort(edges_B[["from_id", "to_id"]].values, axis=1)
edges_B = edges_B.drop_duplicates()
edges_G[["from_id", "to_id"]] = np.sort(edges_G[["from_id", "to_id"]].values, axis=1)
edges_G = edges_G.drop_duplicates()

for each social network, add all the friends an their properties

In [151]:
# add languages and religion to homes
merged_B = pd.merge(homes_B, countries[["alpha_2", "languages", "main_religion"]], how="inner", left_on="country", right_on="alpha_2")
merged_B.columns = ["drop1", "drop2", "country", "drop3", "languages", "main_religion"]
merged_B = merged_B.drop(columns=["drop1", "drop2", "drop3"])
# add friend ids to home locations
merged_B = pd.merge(merged_B, edges_B, how="inner", left_index=True, right_on="from_id").drop(columns=["to_id"])
merged_B.columns = ["country", "languages", "main_religion", "friend"]
# add friend home country
merged_B = pd.merge(merged_B, homes_B["country"], how="inner", left_on="friend", right_index=True)
merged_B.columns = ["country", "languages", "main_religion", "friend", "friend_country"]
# add friend languages and religion
merged_B = pd.merge(merged_B, countries[["alpha_2", "languages", "main_religion"]], how="inner", left_on="friend_country", right_on="alpha_2")
merged_B.columns = ["country", "languages", "main_religion", "friend", "friend_country", "drop1", "friend_languages", "friend_main_religion"]
merged_B = merged_B.drop(columns=["drop1", "friend"])

merged_B

/home/dcelik/.virtualenvs/ada/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,country,languages,main_religion,friend_country,friend_languages,friend_main_religion
0,US,"[English, Spanish, Chinese]",Christianity,US,"[English, Spanish, Chinese]",Christianity
1,US,"[English, Spanish, Chinese]",Christianity,US,"[English, Spanish, Chinese]",Christianity
2,US,"[English, Spanish, Chinese]",Christianity,US,"[English, Spanish, Chinese]",Christianity
3,US,"[English, Spanish, Chinese]",Christianity,US,"[English, Spanish, Chinese]",Christianity
4,US,"[English, Spanish, Chinese]",Christianity,US,"[English, Spanish, Chinese]",Christianity
...,...,...,...,...,...,...
205845,CA,"[English, French, Punjabi]",Christianity,GN,"[French, Pular, Maninka]",Islam
205846,CA,"[English, French, Punjabi]",Christianity,GN,"[French, Pular, Maninka]",Islam
205847,CA,"[English, French, Punjabi]",Christianity,GN,"[French, Pular, Maninka]",Islam
205848,CA,"[English, French, Punjabi]",Christianity,GN,"[French, Pular, Maninka]",Islam


In [152]:
# add languages and religion to homes
merged_G = pd.merge(homes_G, countries[["alpha_2", "languages", "main_religion"]], how="inner", left_on="country", right_on="alpha_2")
merged_G.columns = ["drop1", "drop2", "country", "drop3", "languages", "main_religion"]
merged_G = merged_G.drop(columns=["drop1", "drop2", "drop3"])
# add friend ids to home locations
merged_G = pd.merge(merged_G, edges_G, how="inner", left_index=True, right_on="from_id").drop(columns=["to_id"])
merged_G.columns = ["country", "languages", "main_religion", "friend"]
# add friend home country
merged_G = pd.merge(merged_G, homes_G["country"], how="inner", left_on="friend", right_index=True)
merged_G.columns = ["country", "languages", "main_religion", "friend", "friend_country"]
# add friend languages and religion
merged_G = pd.merge(merged_G, countries[["alpha_2", "languages", "main_religion"]], how="inner", left_on="friend_country", right_on="alpha_2")
merged_G.columns = ["country", "languages", "main_religion", "friend", "friend_country", "drop1", "friend_languages", "friend_main_religion"]
merged_G = merged_G.drop(columns=["drop1", "friend"])
merged_G

/home/dcelik/.virtualenvs/ada/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,country,languages,main_religion,friend_country,friend_languages,friend_main_religion
0,US,"[English, Spanish, Chinese]",Christianity,US,"[English, Spanish, Chinese]",Christianity
1,US,"[English, Spanish, Chinese]",Christianity,US,"[English, Spanish, Chinese]",Christianity
2,US,"[English, Spanish, Chinese]",Christianity,US,"[English, Spanish, Chinese]",Christianity
3,US,"[English, Spanish, Chinese]",Christianity,US,"[English, Spanish, Chinese]",Christianity
4,US,"[English, Spanish, Chinese]",Christianity,US,"[English, Spanish, Chinese]",Christianity
...,...,...,...,...,...,...
656256,JO,"[Arabic, English]",Islam,KZ,"[Kazakh, Russian]",Islam
656257,JO,"[Arabic, English]",Islam,KZ,"[Kazakh, Russian]",Islam
656258,JO,"[Arabic, English]",Islam,KZ,"[Kazakh, Russian]",Islam
656259,JO,"[Arabic, English]",Islam,KZ,"[Kazakh, Russian]",Islam


append them

In [153]:
merged = merged_B.append(merged_G, ignore_index=True)
merged

,country,languages,main_religion,friend_country,friend_languages,friend_main_religion
0,US,"[English, Spanish, Chinese]",Christianity,US,"[English, Spanish, Chinese]",Christianity
1,US,"[English, Spanish, Chinese]",Christianity,US,"[English, Spanish, Chinese]",Christianity
2,US,"[English, Spanish, Chinese]",Christianity,US,"[English, Spanish, Chinese]",Christianity
3,US,"[English, Spanish, Chinese]",Christianity,US,"[English, Spanish, Chinese]",Christianity
4,US,"[English, Spanish, Chinese]",Christianity,US,"[English, Spanish, Chinese]",Christianity
...,...,...,...,...,...,...
862106,JO,"[Arabic, English]",Islam,KZ,"[Kazakh, Russian]",Islam
862107,JO,"[Arabic, English]",Islam,KZ,"[Kazakh, Russian]",Islam
862108,JO,"[Arabic, English]",Islam,KZ,"[Kazakh, Russian]",Islam
862109,JO,"[Arabic, English]",Islam,KZ,"[Kazakh, Russian]",Islam


get a dataframe without entries in the same country

In [154]:
merged_international = merged[merged["country"] != merged["friend_country"]]
merged_international

,country,languages,main_religion,friend_country,friend_languages,friend_main_religion
124237,FI,"[Finnish, Swedish, Russian]",Christianity,US,"[English, Spanish, Chinese]",Christianity
124238,FI,"[Finnish, Swedish, Russian]",Christianity,US,"[English, Spanish, Chinese]",Christianity
124239,FI,"[Finnish, Swedish, Russian]",Christianity,US,"[English, Spanish, Chinese]",Christianity
124240,FI,"[Finnish, Swedish, Russian]",Christianity,US,"[English, Spanish, Chinese]",Christianity
124241,FI,"[Finnish, Swedish, Russian]",Christianity,US,"[English, Spanish, Chinese]",Christianity
...,...,...,...,...,...,...
862106,JO,"[Arabic, English]",Islam,KZ,"[Kazakh, Russian]",Islam
862107,JO,"[Arabic, English]",Islam,KZ,"[Kazakh, Russian]",Islam
862108,JO,"[Arabic, English]",Islam,KZ,"[Kazakh, Russian]",Islam
862109,JO,"[Arabic, English]",Islam,KZ,"[Kazakh, Russian]",Islam


In [155]:
len(merged_international[merged_international["main_religion"] == "Christianity"])

363709

add religions and langauges to homes

In [156]:
homes_B = pd.merge(homes_B, countries[["alpha_2", "languages", "main_religion"]], how="inner", left_on="country", right_on="alpha_2")
homes_G = pd.merge(homes_G, countries[["alpha_2", "languages", "main_religion"]], how="inner", left_on="country", right_on="alpha_2")

homes_B.columns = ["drop1", "drop2", "country", "drop3", "languages", "main_religion"]
homes_G.columns = ["drop1", "drop2", "country", "drop3", "languages", "main_religion"]

homes_B = homes_B.drop(columns=["drop1", "drop2", "drop3"], axis=1)
homes_G = homes_G.drop(columns=["drop1", "drop2", "drop3"], axis=1)

homes_B

/home/dcelik/.virtualenvs/ada/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,country,languages,main_religion
0,US,"[English, Spanish, Chinese]",Christianity
1,US,"[English, Spanish, Chinese]",Christianity
2,US,"[English, Spanish, Chinese]",Christianity
3,US,"[English, Spanish, Chinese]",Christianity
4,US,"[English, Spanish, Chinese]",Christianity
...,...,...,...
50470,LB,"[Arabic, French, English]",Islam
50471,LB,"[Arabic, French, English]",Islam
50472,SN,"[French, Wolof, Pular]",Islam
50473,MR,"[Arabic, Pular, Soninke]",Islam


# Religions

In [157]:
from_list = []
to_list = []
count_list = []
religion_count_international = merged_international.groupby(['main_religion', 'friend_main_religion']).size().sort_values(ascending=False)
for (form_r, to_r), count in religion_count_international.iteritems():
    [from_r, to_r] = sorted([form_r, to_r])
    from_list.append(from_r)
    to_list.append(to_r)
    count_list.append(count)
religion_count_international_df = pd.DataFrame(zip(from_list, to_list, count_list),
                                     columns =['from_religion', "to_religion", "count"]) 

from_list = []
to_list = []
count_list = []
religion_count_international = religion_count_international_df.groupby(["from_religion", "to_religion"])['count'].sum()
for (form_r, to_r), count in religion_count_international.iteritems():
    [from_r, to_r] = sorted([form_r, to_r])
    from_list.append(from_r)
    to_list.append(to_r)
    count_list.append(count)
religion_count_international_df = pd.DataFrame(zip(from_list, to_list, count_list),
                                     columns =['from_religion', "to_religion", "count"]).sort_values("count", ascending=False)

religion_count_international_df

,from_religion,to_religion,count
4,Christianity,Christianity,294429
1,Buddhism,Christianity,49022
6,Christianity,Islam,32722
5,Christianity,Hinduism,1271
3,Buddhism,Islam,646
0,Buddhism,Buddhism,128
8,Islam,Islam,75
2,Buddhism,Hinduism,8
7,Hinduism,Islam,2


divide by possible friendships

In [161]:
relative_count = []

for indx, row in religion_count_international_df.iterrows():
    all_homes = homes_B.append(homes_G, ignore_index=True)
    total = len(all_homes[all_homes["main_religion"] == row["from_religion"]]) * len(all_homes[all_homes["main_religion"] == row["to_religion"]])
    relative_count.append(row["count"] / (total))
    
religion_count_international_df["relative_count"] = relative_count
religion_count_international_df = religion_count_international_df.sort_values("relative_count", ascending=False)
religion_count_international_df

,from_religion,to_religion,count,relative_count
6,Christianity,Islam,32722,0.000056
1,Buddhism,Christianity,49022,0.000044
5,Christianity,Hinduism,1271,0.000023
3,Buddhism,Islam,646,0.000021
4,Christianity,Christianity,294429,0.000014
8,Islam,Islam,75,0.000005
2,Buddhism,Hinduism,8,0.000003
0,Buddhism,Buddhism,128,0.000002
7,Hinduism,Islam,2,0.000001


###### Graph

In [331]:
def edge_thickness(value, values, mini=1.0, maxi=5.0):
    return mini + maxi * math.log(1+value)/math.log(1+max(values))
    

In [338]:
religion_graph = nx.Graph()# Add node for each character

node_list = set(religion_count_international_df["from_religion"].to_list() + religion_count_international_df["to_religion"].to_list())
node_dic = {key:len(all_homes[all_homes["main_religion"] == key]) for key in node_list}


# nodes
for r, count in node_dic.items():
    religion_graph.add_node(r, size=edge_thickness(count, node_dic.values(), maxi=15.0))


# edges
for indx, row in religion_count_international_df.iterrows():
    religion_graph.add_edge(row["from_religion"], 
                            row["to_religion"], 
                            weight = edge_thickness(row["relative_count"], religion_count_international_df["relative_count"].to_list(), maxi=5.0))

pos_ = nx.spring_layout(religion_graph)

In [339]:
def make_edge(x, y, text, width):
    return  go.Scatter(x         = x,
                       y         = y,
                       line      = dict(width = width,
                                   color = 'cornflowerblue'),
                       hoverinfo = 'text',
                       text      = ([text]),
                       mode      = 'lines')

In [340]:
# For each edge, make an edge_trace, append to list
edge_trace = []
for edge in religion_graph.edges():
    
    if religion_graph.edges()[edge]['weight'] > 0:
        char_1 = edge[0]
        char_2 = edge[1]
        x0, y0 = pos_[char_1] 
        x1, y1 = pos_[char_2]
 
    char_1, char_2 = sorted([char_1, char_2])

    text   = "<b>" + str(int(religion_count_international_df[(religion_count_international_df["from_religion"]==char_1) & (religion_count_international_df["to_religion"]==char_2)]["count"])) + '</b>'

    trace  = make_edge([x0, x1, None], [y0, y1, None], text, width = 0.25*religion_graph.edges()[edge]['weight']**1.75)
    edge_trace.append(trace)

In [341]:
# Make a node trace
node_trace = go.Scatter(x         = [],
                        y         = [],
                        text      = [],
                        textposition = "top center",
                        textfont_size = 10,
                        #mode      = 'markers+text',
                        mode      = 'markers',
                        hoverinfo = 'text',
                        marker    = dict(color = [],
                                         size  = [],
                                         line  = None))
# For each node in midsummer, get the position and size and add to the node_trace
for node in religion_graph.nodes():
    x, y = pos_[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    node_trace['marker']['color'] += tuple(['cornflowerblue'])
    node_trace['marker']['size'] += tuple([5*religion_graph.nodes()[node]['size']])
    node_trace['text'] += tuple(['<b>' + node + '</b>' + "<br>" + "# of Practitioners: "+ str(len(all_homes[all_homes["main_religion"]==node]))])

In [342]:
# Customize layout
layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)', # transparent background
    plot_bgcolor='rgba(0,0,0,0)', # transparent 2nd background
    xaxis =  {'showgrid': False, 'zeroline': False}, # no gridlines
    yaxis = {'showgrid': False, 'zeroline': False}, # no gridlines
)# Create figure
fig = go.Figure(layout = layout)# Add all edge traces
for trace in edge_trace:
    fig.add_trace(trace)# Add node trace
fig.add_trace(node_trace)# Remove legend
fig.update_layout(showlegend = False)# Remove tick labels
fig.update_xaxes(showticklabels = False)
fig.update_yaxes(showticklabels = False)# Show figure
fig.show()

### Languages

spit lists into ultiple entries

In [25]:
merged_international_extended = pd.DataFrame([
    [c1, l1, r1, c2, l2, r2] for c1, L1, r1, c2, L2, r2 in merged_international.values
    for l1 in L1 for l2 in L2
], columns=merged_international.columns)

merged_international_extended

,country,languages,main_religion,friend_country,friend_languages,friend_main_religion
0,FI,Finnish,Christianity,US,English,Christianity
1,FI,Finnish,Christianity,US,Spanish,Christianity
2,FI,Finnish,Christianity,US,Chinese,Christianity
3,FI,Swedish,Christianity,US,English,Christianity
4,FI,Swedish,Christianity,US,Spanish,Christianity
...,...,...,...,...,...,...
3187785,JO,Arabic,Islam,KZ,Russian,Islam
3187786,JO,English,Islam,KZ,Kazakh,Islam
3187787,JO,English,Islam,KZ,Russian,Islam
3187788,KW,Arabic,Islam,SM,Italian,Christianity
